### High Level Strategies:
1. Prompting
2. Function calling
3. Tool Calling
4. Json Mode

Import LLM

In [1]:
import os
from langchain_openai import ChatOpenAI

In [2]:
llm = ChatOpenAI(model = "gpt-4o-mini", api_key=os.environ.get("OPENAI_API_KEY"))

In [3]:
llm.invoke("which one is greater 8.11 or 8.9")

AIMessage(content='8.9 is greater than 8.11.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 20, 'total_tokens': 31}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_507c9469a1', 'finish_reason': 'stop', 'logprobs': None}, id='run-6fc3d229-f2b2-4d1c-9f14-48c0c96ce290-0', usage_metadata={'input_tokens': 20, 'output_tokens': 11, 'total_tokens': 31})

In [5]:
conversation = 	'''
Alice: Hi, I bought a laptop from your store recently, but it's really slow and gets hot quickly. Can you help?
John: Hi Alice, I'm sorry to hear that. I'm here to assist. Can you tell me the model and when you purchased it?
Alice: It's the XYZ UltraBook 15, and I bought it two weeks ago.
John: Thanks, Alice. The slowness could be due to background processes or memory issues. Does it happen with specific apps?
Alice: Yes, mostly when I’m using video editing software or have many browser tabs open.
John: That makes sense. Let’s check the Task Manager. Press Ctrl + Shift + Esc to see which processes are using a lot of resources.
Alice: I see the video software and something called “svchost.exe” using a lot of memory.
John: The video software is expected to use a lot, but svchost.exe might just need a restart to clear it up. How about the overheating?
Alice: Yes, it gets really hot after a short time.
John: Overheating can be due to blocked vents or high CPU usage. Try cleaning the vents and placing the laptop on a hard surface. If it continues, check for updates or bring it in for a diagnostic.
Alice: Okay, I'll try that. Thanks for your help, John.
John: You’re welcome, Alice! Let me know if you need anything else. Have a great day!
Alice: Thanks, you too!
'''

Prompting

In [6]:
prompt = f'''You are provided with a conversation between customer and agent. 
        Your task is to extract key information like Topic of conversation, name of customer and agent, sentiment of the call between 1 to 5.
        Always provide results in json format nothing else.
        Conversation: {conversation}'''

In [7]:
from langchain_community.callbacks import get_openai_callback

In [8]:
with get_openai_callback() as cb:
    out = llm.invoke(prompt)
print(out.content)
print(cb)

```json
{
  "Topic": "Laptop performance issues",
  "CustomerName": "Alice",
  "AgentName": "John",
  "Sentiment": 4
}
```
Tokens Used: 405
	Prompt Tokens: 366
	Completion Tokens: 39
Successful Requests: 1
Total Cost (USD): $0.0


In [12]:
type(out.content)

str

Function Calling

In [13]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Summarization(BaseModel):
    """Extract key information from the conversation."""

    Topic: str = Field(description="What was the topic of discussion in the call.")
    Customer: str = Field(description="Name of the customer")
    Agent: str = Field(description="Name of the Agent")
    Sentiment: int = Field(description="What was the sentiment of the customer, from 1 to 5, 1 as most negative and 5 as most positive")


In [14]:
structured_llm = llm.with_structured_output(Summarization)

with get_openai_callback() as cb:
    out = structured_llm.invoke(f"{conversation}")

print(type(out), "\n", out)
print(cb)

<class '__main__.Summarization'> 
 Topic='Laptop performance issues' Customer='Alice' Agent='John' Sentiment=4
Tokens Used: 440
	Prompt Tokens: 420
	Completion Tokens: 20
Successful Requests: 1
Total Cost (USD): $0.0


In [25]:
out.Topic

'Laptop performance issues'

In [15]:
structured_llm.to_json()

{'lc': 1,
 'type': 'constructor',
 'id': ['langchain', 'schema', 'runnable', 'RunnableSequence'],
 'kwargs': {'first': RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x000001DB8EB99100>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x000001DB8EB9A5D0>, root_client=<openai.OpenAI object at 0x000001DB8E7954C0>, root_async_client=<openai.AsyncOpenAI object at 0x000001DB8EB99130>, model_name='gpt-4o-mini', openai_api_key=SecretStr('**********'), openai_proxy=''), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Summarization', 'description': 'Extract key information from the conversation.', 'parameters': {'type': 'object', 'properties': {'Topic': {'description': 'What was the topic of discussion in the call.', 'type': 'string'}, 'Customer': {'description': 'Name of the customer', 'type': 'string'}, 'Agent': {'description': 'Name of the Agent', 'type': 'string'}, 'Sentiment': {'description': 'What wa

In [26]:
from typing_extensions import TypedDict

class Summarization(TypedDict):
    """Extract key information from the conversation."""

    Topic: str = Field(description="What was the topic of discussion in the call.")
    Customer: str = Field(description="Name of the customer")
    Agent: str = Field(description="Name of the Agent")
    Sentiment: int = Field(description="What was the sentiment of the customer, from 1 to 5, 1 as most negative and 5 as most positive")


structured_llm = llm.with_structured_output(Summarization)

with get_openai_callback() as cb:
    out = structured_llm.invoke(f"{conversation}")

print(type(out), "\n", out)
print(cb)

<class 'dict'> 
 {'Topic': 'Laptop performance issues', 'Customer': 'Alice', 'Agent': 'John', 'Sentiment': 2}
Tokens Used: 392
	Prompt Tokens: 372
	Completion Tokens: 20
Successful Requests: 1
Total Cost (USD): $0.0


Tool Calling

In [27]:
from typing import Union


class ConversationalResponse(BaseModel):
    """Respond in a conversational manner. Be kind and helpful."""

    response: str = Field(description="A conversational response to the user's query")


class Response(BaseModel):
    output: Union[Summarization, ConversationalResponse]

structured_llm = llm.with_structured_output(Response)

In [28]:
with get_openai_callback() as cb:
    out = structured_llm.invoke("Who is president of India?")

print(type(out), "\n", out)
print(cb)

<class '__main__.Response'> 
 output=ConversationalResponse(response='The current President of India is Droupadi Murmu. She took office on July 25, 2022, and is the first tribal woman to hold the position.')
Tokens Used: 158
	Prompt Tokens: 117
	Completion Tokens: 41
Successful Requests: 1
Total Cost (USD): $0.0


In [29]:
with get_openai_callback() as cb:
    out = structured_llm.invoke(f"{conversation}")

print(type(out), "\n", out)
print(cb)

<class '__main__.Response'> 
 output={'Topic': 'Laptop Performance Issues', 'Customer': 'Alice', 'Agent': 'John', 'Sentiment': 4}
Tokens Used: 437
	Prompt Tokens: 415
	Completion Tokens: 22
Successful Requests: 1
Total Cost (USD): $0.0


Json Mode

In [30]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import PromptTemplate

In [31]:
class Summarization(BaseModel):
    """Extract key information from the conversation."""

    Topic: str = Field(description="What was the topic of discussion in the call.")
    Customer: str = Field(description="Name of the customer")
    Agent: str = Field(description="Name of the Agent")
    Sentiment: int = Field(description="What was the sentiment of the customer, from 1 to 5, 1 as most negative and 5 as most positive")


parser = JsonOutputParser(pydantic_object=Summarization)

In [32]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "Extract key information from the conversation.", "properties": {"Topic": {"title": "Topic", "description": "What was the topic of discussion in the call.", "type": "string"}, "Customer": {"title": "Customer", "description": "Name of the customer", "type": "string"}, "Agent": {"title": "Agent", "description": "Name of the Agent", "type": "string"}, "Sentiment": {"title": "Sentiment", "description": "What was the sentiment of the customer, from 1 to 5, 1 as most negative and 5 as most positive", "type": "integer"}}, "required": 

In [33]:
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [35]:
print(prompt)

input_variables=['query'] partial_variables={'format_instructions': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"description": "Extract key information from the conversation.", "properties": {"Topic": {"title": "Topic", "description": "What was the topic of discussion in the call.", "type": "string"}, "Customer": {"title": "Customer", "description": "Name of the customer", "type": "string"}, "Agent": {"title": "Agent", "description": "Name of the Agent", "type": "string"}, "Sentiment": {"title": "Sentiment", "description": "What was the sentiment of the customer, from 1 to 5, 1

In [36]:
chain = prompt | llm | parser

In [37]:
with get_openai_callback() as cb:
    out = chain.invoke(f"{conversation}")

print(out)
print(cb)

{'Topic': 'Laptop performance issues', 'Customer': 'Alice', 'Agent': 'John', 'Sentiment': 3}
Tokens Used: 628
	Prompt Tokens: 591
	Completion Tokens: 37
Successful Requests: 1
Total Cost (USD): $0.0


In [38]:
import json
json.dumps(out)

'{"Topic": "Laptop performance issues", "Customer": "Alice", "Agent": "John", "Sentiment": 3}'